In [2]:
import fitz
import os
from pathlib import Path
import spacy
import pprint
from sentence_transformers import SentenceTransformer, util
import time
import numpy as np

In [3]:
model_name = 'neuralmind/bert-base-portuguese-cased'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
start = time.time()

In [5]:
caminho_do_arquivo = Path("/content/tela_sistema_2.pdf")

### Extraindo todas as frases

In [6]:
#Extraindo texto do arquivo
doc = fitz.open(caminho_do_arquivo)
page = doc.load_page(0)  # Load the first page

text = page.get_text("text")

text = text.split("\n")

### Pegando as posições das frases

In [7]:
texts = []
data = page.get_text("dict")

for  i,block in enumerate(data["blocks"]):
    if data["blocks"][i]["type"] == 0:
        for j, line in enumerate(data["blocks"][i]["lines"]):
            text = data["blocks"][i]["lines"][j]["spans"][0]["text"]
            bbox = data["blocks"][i]["lines"][j]["spans"][0]["bbox"]
            bbox = tuple(round(item, 2) for item in bbox)
            texts.append((text, bbox))

In [8]:
texts

[('Consulta de Cobrança', (204.0, 0.0, 323.39, 16.49)),
 ('Pesquisar por:', (6.0, 17.0, 64.02, 29.37)),
 ('Tipo:', (103.0, 17.0, 125.23, 30.74)),
 ('Sistema:', (335.0, 16.0, 373.9, 29.74)),
 ('Contrato:', (464.0, 16.0, 505.13, 29.74)),
 ('Todos', (337.0, 27.0, 364.79, 40.74)),
 ('Todos', (467.0, 27.0, 494.79, 40.74)),
 ('V', (568.0, 28.0, 573.34, 38.99)),
 ('Buscar', (605.0, 25.0, 636.12, 38.74)),
 ('CLIENTE', (8.0, 27.0, 46.01, 39.37)),
 ('CPF', (107.0, 27.0, 127.0, 40.74)),
 ('Operação Selecionada', (6.0, 49.0, 117.89, 64.11)),
 ('Sistema', (465.0, 68.0, 501.12, 81.74)),
 ('CONSIGNADO', (465.0, 80.0, 526.51, 92.37)),
 ('VIr. Parc.', (464.0, 94.0, 505.68, 107.74)),
 ('2.372,64', (465.0, 107.0, 503.92, 120.74)),
 ('Cidade: Mozarlândia U.F .: GO CEP: 76709970', (6.0, 387.0, 195.58, 399.37)),
 ('Dados para Contato', (8.0, 403.0, 106.44, 418.11)),
 ('Resumo', (11.0, 478.0, 48.23, 491.74)),
 ('Cobradora', (58.0, 500.0, 105.24, 513.74)),
 ('Nenhuma', (356.0, 500.0, 399.35, 513.74)),
 ('Tele

In [9]:
end = time.time()

In [10]:
end - start

0.06997227668762207

In [11]:
candidates = []
for text in enumerate(texts):
  candidates.append(text)

In [17]:
#Cálculo Similaridade

test_text = ["Valor Parcelado", "Cidade", "Dinheiro", "Sistema"]
candidate_embeddings = model.encode(candidates, convert_to_tensor=True)
for text in test_text:
  query_embedding = model.encode(text, convert_to_tensor=True)

  cosine_scores = util.cos_sim(query_embedding, candidate_embeddings)

  ranked_results = sorted(
      zip(candidates, cosine_scores[0].tolist()),
      key=lambda x: x[1],
      reverse=True
  )

  print(f"Query: {test_text}\n")
  print("--- Resultados Classificados ---")
  for candidate, score in ranked_results:
      print(f"{score:.4f}\t{candidate}")

Query: ['Valor Parcelado', 'Cidade', 'Dinheiro', 'Sistema']

--- Resultados Classificados ---
0.4400	(0, ('Consulta de Cobrança', (204.0, 0.0, 323.39, 16.49)))
0.4344	(16, ('Cidade: Mozarlândia U.F .: GO CEP: 76709970', (6.0, 387.0, 195.58, 399.37)))
0.4193	(26, ('Negativado SPC?', (29.0, 548.0, 108.48, 561.74)))
0.4081	(24, ('Negativado SERASA?', (11.0, 535.0, 110.49, 548.74)))
0.3986	(4, ('Contrato:', (464.0, 16.0, 505.13, 29.74)))
0.3985	(17, ('Dados para Contato', (8.0, 403.0, 106.44, 418.11)))
0.3939	(13, ('CONSIGNADO', (465.0, 80.0, 526.51, 92.37)))
0.3917	(30, ('Status Notificação', (23.0, 576.0, 87.47, 586.99)))
0.3913	(19, ('Cobradora', (58.0, 500.0, 105.24, 513.74)))
0.3882	(27, ('Não', (367.0, 549.0, 385.34, 562.74)))
0.3852	(23, ('Cobradora', (57.0, 521.0, 104.24, 534.74)))
0.3805	(14, ('VIr. Parc.', (464.0, 94.0, 505.68, 107.74)))
0.3785	(25, ('Não', (367.0, 535.0, 385.34, 548.74)))
0.3769	(28, ('Status Jurídico', (37.0, 563.0, 103.14, 576.74)))
0.3753	(29, ('Nenhum', (358

In [15]:
cosine_scores = util.cos_sim(query_embedding, candidate_embeddings)

ranked_results = sorted(
    zip(candidates, cosine_scores[0].tolist()),
    key=lambda x: x[1],
    reverse=True
)

print(f"Query: {test_text}\n")
print("--- Resultados Classificados ---")
for candidate, score in ranked_results:
    print(f"{score:.4f}\t{candidate}")

Query: ['Valor Parcelado', 'Cidade', 'Dinheiro', 'Sistema']

--- Resultados Classificados ---
0.4400	(0, ('Consulta de Cobrança', (204.0, 0.0, 323.39, 16.49)))
0.4344	(16, ('Cidade: Mozarlândia U.F .: GO CEP: 76709970', (6.0, 387.0, 195.58, 399.37)))
0.4193	(26, ('Negativado SPC?', (29.0, 548.0, 108.48, 561.74)))
0.4081	(24, ('Negativado SERASA?', (11.0, 535.0, 110.49, 548.74)))
0.3986	(4, ('Contrato:', (464.0, 16.0, 505.13, 29.74)))
0.3985	(17, ('Dados para Contato', (8.0, 403.0, 106.44, 418.11)))
0.3939	(13, ('CONSIGNADO', (465.0, 80.0, 526.51, 92.37)))
0.3917	(30, ('Status Notificação', (23.0, 576.0, 87.47, 586.99)))
0.3913	(19, ('Cobradora', (58.0, 500.0, 105.24, 513.74)))
0.3882	(27, ('Não', (367.0, 549.0, 385.34, 562.74)))
0.3852	(23, ('Cobradora', (57.0, 521.0, 104.24, 534.74)))
0.3805	(14, ('VIr. Parc.', (464.0, 94.0, 505.68, 107.74)))
0.3785	(25, ('Não', (367.0, 535.0, 385.34, 548.74)))
0.3769	(28, ('Status Jurídico', (37.0, 563.0, 103.14, 576.74)))
0.3753	(29, ('Nenhum', (358